# A Physicist Plans a Party (Comparing Prices with Monte Carlo Simulation)

> Adam Lyon, January 2018

We're planning our daughter's Bat Mitzvah. Banquet halls offer different catering and beverage plans that we need to evaluate. It's hard to do this without knowing *a priori* who is coming and what everyone will drink. So we have to guess. A Monte Carlo simulation is one way to do the guessing. Instead of trying to guess particular situations, we'll try to guess *all* situations (or at least a large sample). We create many, many Bat Mitzvahs randomly selecting guest count and drink selections. We can then apply different the pricing rules to the parties. That will create distributions of total prices and other results that we can compare to each other. 

In this notebook, we'll examine two banquet halls WT and cpr (names changed out of paranoia). 

Speed is important here as we want to make as many guesses as we can. I'm going to try to use the [Julia Language](https://julialang.org) as it is just-in-time compiled. Exploiting [parallelization](https://docs.julialang.org/en/stable/manual/parallel-computing) is easier in Julia than in other languages. My laptop has four [hyperthreaded](https://en.wikipedia.org/wiki/Hyper-threading) cores, so I can get eight simultaneous threads going. That's a free factor of ~8 in speed! With the current code here I can generate a million Bat Mitvah's in a minute. 

Click on "Results" in the Table of Contents below to skip to the results.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Configure-Julia" data-toc-modified-id="Configure-Julia-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configure Julia</a></span></li><li><span><a href="#Party-Parameters" data-toc-modified-id="Party-Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Party Parameters</a></span><ul class="toc-item"><li><span><a href="#Define-Guests" data-toc-modified-id="Define-Guests-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Define Guests</a></span></li><li><span><a href="#Define-drinks" data-toc-modified-id="Define-drinks-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Define drinks</a></span></li><li><span><a href="#Define-costs" data-toc-modified-id="Define-costs-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Define costs</a></span></li></ul></li><li><span><a href="#Make-Parties" data-toc-modified-id="Make-Parties-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make Parties</a></span><ul class="toc-item"><li><span><a href="#Run-single-threaded" data-toc-modified-id="Run-single-threaded-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run single threaded</a></span></li><li><span><a href="#Restore-from-file" data-toc-modified-id="Restore-from-file-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Restore from file</a></span></li><li><span><a href="#Run-in-Parallel" data-toc-modified-id="Run-in-Parallel-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Run in Parallel</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#Check-coverage" data-toc-modified-id="Check-coverage-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Check coverage</a></span></li><li><span><a href="#Distributions-of-guests" data-toc-modified-id="Distributions-of-guests-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Distributions of guests</a></span></li><li><span><a href="#Distributions-of-drinks" data-toc-modified-id="Distributions-of-drinks-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Distributions of drinks</a></span></li><li><span><a href="#Cost-results" data-toc-modified-id="Cost-results-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Cost results</a></span></li><li><span><a href="#Cost---people-correlation?" data-toc-modified-id="Cost---people-correlation?-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Cost - people correlation?</a></span></li><li><span><a href="#Explore-cpr-where-package-plan-is-favored" data-toc-modified-id="Explore-cpr-where-package-plan-is-favored-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Explore cpr where package plan is favored</a></span></li></ul></li><li><span><a href="#Back-Matter" data-toc-modified-id="Back-Matter-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Back Matter</a></span></li></ul></div>

## Configure Julia

In [1]:
addprocs() ; nworkers() # Use all available threads on the machine

8

In [2]:
# Load packages ... @everywhere loads them on all the worker threads
#  This can take a long time to run
@everywhere begin
    using Base.Test
    using Lazy: @>, @>>, @_
    using Distributions
    using Plots
    using DataFrames
    using DataFramesMeta
    using StatPlots
end

A note about the `@>` syntax you'll see below as it will likely look unfamiliar. The [Lazy.jl](https://github.com/MikeInnes/Lazy.jl) package has the `@>` and `@>>` (and other) macros to create a pipeline of commands. `@>` threads the result of a function into the first argument position of the next function on the line. `@>>` threads the result of a function into the *last* argument position of the next function (needed rarely). For example,

```julia
mean( @select(x, :nA ) )  # Take the mean of the :nA column of the data frame x
@> x @select(:nA) mean    # This line does the same as above

ndupl = sum( nonunique( @select(x, :nA, :nT, :nC, :nM, :nW, :nSA, :nSK) ) )
ndupl = @> x @select(:nA, :nT, :nC, :nM, :nW, :nSA, :nSK) nonunique sum()  # Same as line above

printfmtln("{1:.2f}%", ndupl/nrow(x)*100 )
@>> ndupl/nrow(x)*100 printfmtln("{1:.2f}%")  # Same as line above
```

I find the threading macros make complex expressions easier to read as you can follow the flow from left to right instead of from inner parentheses to outer. 

In [3]:
srand()   # Seed the global random number generator

MersenneTwister(UInt32[0x78aa194a, 0xf33d9745, 0xb4d6e9ca, 0xcbcd8a84], Base.dSFMT.DSFMT_state(Int32[-48328105, 1073687623, -1040152445, 1072777011, -373579260, 1072970717, -1116174228, 1073550378, -519469233, 1073610947  …  1652690625, 1072808875, 570706538, 1073679198, -1098959792, -730144734, -155807941, 644332446, 382, 0]), [1.45228, 1.79759, 1.4128, 1.00926, 1.72241, 1.62337, 1.40036, 1.99739, 1.50656, 1.4634  …  1.22411, 1.22186, 1.57495, 1.94314, 1.36692, 1.73801, 1.82258, 1.88827, 1.76697, 1.70464], 382)

*Assumptions and limitations:*

* We are going to ignore correlations between guests. For example, many guests come in families. We're going to ignore that for now.
* We'll assume that no one drinks beer

Party parameters from the latest [guest list](https://docs.google.com/spreadsheets/d/1XnHJmx3Ke0D1UaFNWex0c4MNv9CawyiZPqVUIqkzePY/edit#gid=0)

In [4]:
function printNicely(x) 
    """
    Julia prints unsigned integers in hex by default. Print in decimal instead
    """
    [println(aNum) for aNum in x]
    nothing
end

printNicely (generic function with 1 method)

## Party Parameters

Here is where we define the parameters of the parties

### Define Guests

In [5]:
# Ranges of guests
@everywhere begin
    const nParties    = 1_000_000  # How many parties to generate? 
                                   #  Can use _ as grouping marker (e.g. 1,000,000)
                                   #  Perhaps change to 100_000 if running on mybinder
    const adultRange  = (41, 71)   # Adult food rate and drinks alcohol
    const teenRange   = (18, 27)   # 13 and over (adult food rate, but no alcohol)
    const childRange  = (11, 22);  # Child fee 
end

In [6]:
@everywhere function randIntInRange(range::Tuple{Int, Int})
    """
    Return a flat random distribution within the given range
    """
    
    floor(Int, rand()*(range[2]-range[1]+1)+range[1])
end

@everywhere function randomsInRanges(ranges...)
    """
    Pull a random number for each range. The output is in the same order of the ranges
    """
    [randIntInRange(r) for r in ranges]
end

### Define drinks

We need to determine how many drinks the guests are going to have. Here are the categories of drinks
* Soda
* House wine
* Mixed drink (top shelf)

Let's assume the following:

__Drinking Behavior A__

* On average, each person has a drink an hour. Some people may drink much less. Others make drink a little more. 
* Let's do the number of drinks for adults as a gaussian with mean of 4 and sd of 2. We do not allow more than 5 drinks. 
* For adults, two of their drinks have a 20% chance of being a mixed drink, 50% wine, 30% soda
* For adults, the third drink has 50% wine, 50% soda, 0% mixed drink
* For adults, the other drinks have 10% wine, 90% soda
* For kids, number of sodas is a gaussian with mean of 5 and sd of 1

Here's what a Gaussian $\mu=4; \sigma=2$ looks like (with integers; note that I've centered the bins on the integers)

In [7]:
gr()
niceHistBins(low, high) = low-0.5:high+0.5
niceHistBins(t::Tuple) = niceHistBins(t[1], t[2])
@_ round.(Int, rand(Normal(4, 2),10000)) filter(x -> x <= 5, _) histogram(_, nbin=niceHistBins(0, 8), normalize=true, legend=nothing, line=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25

Here's what a Gaussian with $\mu=5; \sigma=2$ looks likes (with integers)

In [8]:
@> round.(Int, rand(Normal(5, 2),10000))  histogram(nbin=niceHistBins(1, 9), normalize=true, legend=nothing, line=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20

In [9]:
@everywhere function drinksForProb(vs::Array{Float64,1}, probMixed::Float64, probWine::Float64, probSoda::Float64)
    """
    Given a flat [0,1] random number array vs, determine if the drink is a mixed drink, wine, or soda
    according to the probabilities.
    Returns [nMixed, nWine, nSoda, 0]  # We always return soda in the third element 
    """
    @assert probMixed + probWine + probSoda ≈ 1.0
    
    nDrinks = zeros(Int, 4)
    
    nDrinks[1] += @> vs .<= probMixed sum                          # Mixed drinks
    nDrinks[2] += @> probMixed .< vs .<= probWine+probMixed sum    # Wine
    nDrinks[3] += @> probWine+probMixed .< vs sum                  # Sodas
    
    return nDrinks
end

In [10]:
# Some tests to be sure I did this right
@test drinksForProb([0.5, 0.1, 0.95], 0.7, 0.2, 0.1) == [2, 0, 1, 0]
@test drinksForProb([0.5, 0.8, 0.95], 0.7, 0.2, 0.1) == [1, 1, 1, 0]
@test drinksForProb([0.95, 0.95, 0.95], 0.7, 0.2, 0.1) == [0, 0, 3, 0]
@test drinksForProb([0.95, 0.95, 0.95], 0.8, 0.2, 0.0) == [0, 3, 0, 0]

Test Passed

In [11]:
@everywhere function drinkingBehaviorA(g::Array{Int,1})
    """
    Implement Drinking Behavior A (see above)
    g has the form [# of adults, # of teens, # of kids]
    """
    
    nDrinks = zeros(Int, 4)  # [nMixed, nWine, nSoda_adults, nSoda_kids]
    
    # Adult drinks
    nAdultDrinks = @>> round.(Int, rand(Normal(4, 2), g[1]) ) filter( x -> x <= 5)
    nDrinks += @> rand(@> nAdultDrinks .>= 1 sum) drinksForProb(0.2, 0.5, 0.3)   # Drink 1
    nDrinks += @> rand(@> nAdultDrinks .>= 2 sum) drinksForProb(0.2, 0.5, 0.3)   # Drink 2
    nDrinks += @> rand(@> nAdultDrinks .>= 3 sum) drinksForProb(0.0, 0.5, 0.5)   # Drink 3
    for i in 4:maximum(nAdultDrinks)                                             # Drinks 4, ...
        nDrinks += @> rand(@> nAdultDrinks .>= i sum) drinksForProb(0.0, 0.10, 0.90)
    end
    
    # Kids drinks (# of sodas)
    nDrinks[4] += @> round.(Int, rand(Normal(5, 2), g[2] + g[3]) ) sum
    
    nDrinks
end

### Define costs

With the number of people and the number of drinks, we can try several different cost plans for the party.  Below are functions that create data frame entries for each "party". 

In [12]:
@everywhere begin

    function makeCommonEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Make the common entries
               w = worker thread ID,   nA = #Adults,       nT = #Teens, nC = #Children
              nM = #Mixed drinks,      nW = #Wine drinks, 
             nSA = #sodas for adults, nSK = #sodas for kinds (teens+children)
        """
        DataFrame(w=myid(), nA=g[1], nT=g[2], nC=g[3], nM=d[1], nW=d[2], nSA=d[3], nSK=d[4])
    end

    function wtEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Costing for the WT Banquet Hall
           Wfood   = Cost for food,          WdrinkT = drink cost on tab plan,     wDrinkP = drink costs on package plan
           WtotalT = total cost on tab plan, WtotalP = total cost on package plan,    WT_P = total tab - total package
        """

        wtFood(g) = 23.0(g[1]+g[2]) + 13.0g[3]                   #  Food: $23/adult, $13/child
        wtDrinkTab(g, d) = 9.0d[1] + 7.5d[2] + 3d[3] + 3d[4]     #  Drink tab: Mixed=$9, Wine=$7.50, Soda=$3
        wtDrinkPkg(g, d) = 20.0(g[1] + g[2] + g[3])              #  Drink package: $20/person

        food = wtFood(g)
        drinkT = wtDrinkTab(g, d)
        drinkP = wtDrinkPkg(g, d)

        totalT = (1.0 + 0.2 + 0.0975)*(food + drinkT)                    # Tax is 9.75% and service is 20%
        totalP = (1.0 + 0.2 + 0.0975)*(food + drinkP)

        DataFrame(Wfood=food, WdrinkT=drinkT, WdrinkP=drinkP, WtotalT=totalT, WtotalP=totalP, WT_P=totalT-totalP)
    end

    function cprEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Costing for the cpr Banquet Hall
           Cfood   = Cost for food,          CdrinkT = drink cost on tab plan,     CDrinkP = drink costs on package plan
           CtotalT = total cost on tab plan, CtotalP = total cost on package plan,    CT_P = total tab - total package
        """

        cprFood(g) = 27.0*(g[1] + g[2]) + 15.0*g[3]                        # Food is $27/adult, $15/child
        cprDrinkTab(g, d) = 9.0*d[1] + 7.0*d[2] + 1.0*d[3] + 1.0*d[4]      # Drink tab: Mixed=$9, Wine=$7, Soda=$1
        cprDrinkPkg(g, d) = 3.0*(g[1]+g[2]+g[3]) + 9.0*d[1] + 7.0*d[2]     # Unlimited soda service $3/person 

        food = cprFood(g)
        drinkT = cprDrinkTab(g, d)
        drinkP = cprDrinkPkg(g, d)

        totalT = (1.0 + 0.2 + 0.1075)*(food + drinkT) + 500.0
        totalP = (1.0 + 0.2 + 0.1075)*(food + drinkP) + 500.0

        DataFrame(Cfood=food, CdrinkT=drinkT, CdrinkP=drinkP, CtotalT=totalT, CtotalP=totalP, CT_P=totalT-totalP)
    end
    
end

I have a spreadsheet with some pre-worked out examples. Let's check them. 

In [13]:
wtEntry([50, 25, 15], [50, 150, 100, 30])

,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P
1,1920.0,1965.0,1800.0,5040.79,4826.7,214.087


In [14]:
cprEntry([50, 25, 15], [50, 150, 100, 30])

,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,2250.0,1630.0,1770.0,5573.1,5756.15,-183.05


## Make Parties

Time to party!

In [15]:
@everywhere function makeParty()
    """Make one party"""
    
    # Determine guests
    g = randomsInRanges(adultRange, teenRange, childRange)
    
    # Determine drinks
    d = drinkingBehaviorA(g)
    
    # Common entry
    c = makeCommonEntry(g, d)
    
    # Cost at WT
    c = hcat(c, wtEntry(g, d))
        
    # Cost at CPR
    c = hcat(c, cprEntry(g, d))
    
    c
    
end 

In [35]:
@time makeParty()

  0.000237 seconds (482 allocations: 120.328 KiB)


,w,nA,nT,nC,nM,nW,nSA,nSK,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,1,58,27,12,15,63,59,192,2111.0,1360.5,1940.0,4504.27,5256.17,-751.901,2475.0,827.0,867.0,4817.36,4869.66,-52.3


**Choose from three options for how to proceed**

### Run single threaded

In [36]:
x = makeParty()     # Careful - this can take a long time! 
                    # Maybe run the parallel cell instead!
@time for i in 1:nParties-1
    append!(x, makeParty())
end

 31.212703 seconds (65.01 M allocations: 14.572 GiB, 11.30% gc time)


### Restore from file

In [40]:
using JLD
function dataFromCache(filename)
    if isfile(filename)
        return JLD.load(filename)["x"]
    end
    return nothing
end

dataFromCache (generic function with 1 method)

In [41]:
x = dataFromCache("party1M.jld");

### Run in Parallel

In [18]:
@time x = @parallel (append!) for i = 1:nworkers()
    y = makeParty()
    for i in 1:nParties/nworkers() - 1
        append!(y, makeParty())
    end
    y
end ;

 61.228146 seconds (16.08 k allocations: 294.724 MiB, 0.26% gc time)


In [41]:
using JLD
@save "party1M.jld" x

## Results

The result is a [DataFrame](http://juliadata.github.io/DataFrames.jl/stable/) with information about each party. Let's just look at the top few rows.

In [19]:
head(x)

,w,nA,nT,nC,nM,nW,nSA,nSK,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,2,49,20,16,19,61,53,193,1795.0,1366.5,1700.0,4102.05,4534.76,-432.716,2103.0,844.0,853.0,4353.2,4364.97,-11.7675
2,2,65,24,11,17,45,76,167,2190.0,1219.5,2000.0,4423.83,5436.52,-1012.7,2568.0,711.0,768.0,4787.29,4861.82,-74.5275
3,2,50,19,15,11,58,70,176,1782.0,1272.0,1680.0,3962.56,4491.94,-529.38,2088.0,751.0,757.0,4211.99,4219.84,-7.845
4,2,67,22,20,21,65,71,191,2307.0,1462.5,2180.0,4890.93,5821.88,-930.956,2703.0,906.0,971.0,5218.77,5303.75,-84.9875
5,2,69,21,20,26,69,92,209,2330.0,1654.5,2200.0,5169.89,5877.67,-707.786,2730.0,1018.0,1047.0,5400.51,5438.43,-37.9175
6,2,67,19,17,16,68,79,181,2199.0,1434.0,2060.0,4713.82,5526.05,-812.235,2577.0,880.0,929.0,5020.03,5084.09,-64.0675


How many parties did we get?

In [20]:
using Formatting

@> nrow(x) format(commas=true) println

1,000,000


How many rows are duplications? If there are a lot, that could mean we've oversampled. 

In [21]:
# Look for duplicate rows. We need to remove the w column as that's the worker node ID
ndupl = @> x @select(:nA, :nT, :nC, :nM, :nW, :nSA, :nSK) nonunique sum()

279

In [22]:
@>> ndupl/nrow(x)*100 printfmtln("{1:.2f}%")

0.03%


I think this is ok.

### Check coverage

We need to look at coverage (did we cover a large sample of possibilities?). How many possible set of guests should we have (assuming no correlations)?

In [23]:
tdiff(t::Tuple{Int,Int}) = t[2]-t[1]+1
totalGuestSetsExpected = tdiff(adultRange) * tdiff(teenRange) * tdiff(childRange)

3720

How many sets of guests did we generate?

In [24]:
gs = @> x @select(:nA, :nT, :nC)          # Select out the columns involving guests
@> nrow(gs) format(commas=true) println

1,000,000


In [25]:
totalGuestSets = @> gs unique nrow    # Get the unique set (remove all duplicates)

3720

In [26]:
# They should agree with what we expect
@test totalGuestSetsExpected == totalGuestSets

Test Passed

We now want to see how many samples there are for each guest set. To do this, we'll make a unique hash for each row of guest sets and see how often each appears in the full set of parties. 

In [27]:
gsHashes = [ hash(r) for r in eachrow(gs) ]
nHashes = @> gsHashes unique length

3720

In [28]:
@test nHashes == totalGuestSets # This should match the total number of guest sets

Test Passed

In [29]:
using FreqTables

In [30]:
gsFreq = freqtable(gsHashes) # Determine how often a hash (guest set) appears 
gsFreq[1:5]  # Just show the first five

5-element Named Array{Int64,1}
Dim1               │ 
───────────────────┼────
0x00023f9b13bb2633 │ 237
0x0012f8d42691d831 │ 249
0x00145e2cafb86bff │ 269
0x00321dc9a184eafe │ 264
0x003d384ad9002082 │ 270

In [31]:
histogram(gsFreq.array, title="Number of drink sets for guest set", xlab="Number of drink sets", legend=nothing, line=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 225 
 
 
 250 
 
 
 275 
 
 
 300 
 
 
 325 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 Number of drink sets for guest set 
 
 
 Number of drink sets

In [32]:
length(gsFreq)

3720

In [33]:
@> sum(gsFreq) format(commas=true) println  # Should be the number entries in the whole list

1,000,000


In [34]:
mean(gsFreq), minimum(gsFreq), maximum(gsFreq)

(268.81720430107526, 213, 327)

The [histogram](https://en.wikipedia.org/wiki/Histogram) above is a distribution of the number of drink sets per guest set (this is the distribution of the number of times a guest set appears in the list, presumably with different drinks). In order to cover a large number of drink situations, we want this distribution to go out far (going out too far leads to duplication and unnecessary parties). With 100,000 parties, the plot has a mean of 27. With 1M parties we get a mean of 268. The Gaussian distribution in the plot is expected from the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) [it works!]. Note that this plot used to look stratified with three distinct peaks, and the distribution of guests had the edge bins at half the height as the rest. That was because I originally had the random number generation for the number of guests calling `round` instead of `floor`. Using `round` means that the values at the ends of the range get hit only half as often -- a good catch by this test. 

### Distributions of guests 
Who did we invite? Let's look at [histograms](https://en.wikipedia.org/wiki/Histogram) that display distributions of interesting results from the parties. For the number of guests, we generated flat distributions and that's what we should see.

In [38]:
h1 = @df x histogram( :nA, legend=false, nbins=niceHistBins(adultRange), title="#Adults", line=nothing)
h2 = @df x histogram( :nT, legend=false, nbins=niceHistBins(teenRange), title="#Teens", line=nothing)
h3 = @df x histogram( :nC, legend=false, nbins=niceHistBins(childRange), title="#Children", line=nothing)
plot(h1, h2, h3, layout=(3,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 #Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 #Teens 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 #Children

Are there any correlations?

In [39]:
p1 = @df x histogram2d( :nA, :nT, nbin=15, xlab="Number of adults", ylab="Number of teens")
p2 = @df x histogram2d( :nA, :nC, nbin=15, xlab="Number of adults", ylab="Number of children")
p3 = @df x histogram2d( :nT, :nC, nbin=15, xlab="Number of teens", ylab="Number of children")
plot(p1, p2, p3, layout=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 Number of adults 
 
 
 Number of teens 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAErElEQVR4nO3dO2+cRRiG4Xe+b3Zt
7/pEEhOZRJGIUEApEAJRhII2Eh3/k4aChoZDR0MBQhAZJVIOTuIQJ1lns8eZocgfeNJYT3Ff9auZ
1erWV41mUu4PWoheHn0tzfVnJ+qSsT74SFvzxX1prm1uy3un+Zk0V0cXxL335L2jFW0sj+Ql+8lD
be7abWluffyzvHe30P7LsntFW0/eGThHhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlL
hAlLhAlLKfcX5WNvr/75Spqr4wP5B3SzU23NvKEtqM5FRESSptrosrbcaqrvvHghzW1d/UZec/bg
B23v9Vyaa+9/Ie+dTn7XBrteG5N3Bs4RYcISYcISYcISYcISYcISYcISYcISYcISYcISYcISYcIS
YcISYcJSbuKtYxERg+vfSnOLJ7/Ka7Z+oA3moTSWFu9w9OzwljRXJ0fSXH96T9677B5Kc7MnP8lr
xmBLGqu7H0pzKWnHAiP0o3Tlwg1pji8mLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEm
LBEmLBEmLGXxwrOIiFg8+lEbHO7Ia6bZQhtsVV5T1Z7+ps3tXJXmyo5+y10k7ZuQxNvw3g5rN6nF
SrztbXNf31sNSTxmyRcTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlnIp
r+Xhtn1FGywrec2yq605PP5Dmlsefirvnfc+0QaPf5HG2rs8sJrW+qyo7l/Xtn79UJubPZf3bsOx
tmbRjtzxxYQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoSlPMyX5OH05kRb9OBL
/Rfc+U4aKyPtgqdO/I0REaXMtDXF01J1X3ujMSIizbXLstrGtr7m9LG2pviWZKy1/yciont+R9t7
c09bT94ZOEeECUuECUuECUuECUuECUuECUuECUuECUuECUuECUuECUuECUuECUu5NPEtx4iI/Y+l
sdVSf/8w7X2gzYlvSdYd8eKviEjz/6S5weN/pbly+TN571iJF1tV/fKtNtjS1pweawuWpbx3HWvv
aLaNi9IcX0xYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYIkxYyrnTjkpFRLQ3
2nGpfHpXXrMONqW5bq29QVjfuyHvnZYPpLn5zdvSXP/sT3lv9ZhYHWq33L39AeJbluupOKff9paW
E22waTfn8cWEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEpVyqfttbP3kk
zZWxdqNXRERaaUer0uxMmutO/5b3bsORtqb4eGgVH/CMiGjdQJ5Vpck9bU69Qa4WeW/1prnIY2mM
LyYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYs5dxrx5AiIupQO9rUXfpc
XrM7+l6aW127Ja7Y5L1DPHrWPftL21k9+hX6EcKWxRvcIiKJj5K2pH6P9P+ym2oPxlZxTb6YsESY
sESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsPQ/aDbfqNRwcKYAAAAASUVORK5CYII=
" transform="translate(55, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAACUCAYAAACeJ3mSAAABW0lEQVRogd3a23EDQQhE0V57nH8S
ztEvBpzE+aCkAG7RFxitdvRUf07A5/SU4OT0/UagRqBZF20UKAzUyFHqC4FUtOf+GFD6GtBTvwh0
EWhhtPpTIOaoDWifozCQc8TOo23RUmjXcpkjVdE+R0ZRDnqGyIlRlBOjKGfqQaD7ZkDpdRXVuwHN
qGiFos1V0RY6UhWpOWom260IqqiVo2kme1vXnCO1Ii6a6tplK+IcqTlSjli0y3ZtH4i1n012o8l2
XdvnaJhs5ohwYEXMEWv/XXceQUeEk9Nsstd1beEcsZ8QBJOcUg/s+xzB9hMOjFZsjlw0BHLRCCY5
pV6NwRUhHPkQ8bog6MiQFjqCxwhyVEGgYRWtc3TRS//TSnYNqohFKwUaVtGDLqEK3UItjHZVtAq6
px10L+a6dlW0ZiC1/e5fYy8cbVg0doygFdnnKIOijZqjQcfIk3yQL/9/fmk1aCz3ydQAAAAASUVO
RK5CYII=
" transform="translate(232, 12)"/>
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 Number of adults 
 
 
 Number of children 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAFVElEQVR4nO3dvW4cVRjG8XPmY3f8
tfF6Y8cEGycxBQKFSNR0NDTABVByAbRcERUSBT1QAA1EgAQiRAHHDnFYLNvrXTv2emfOUHADTyRk
PcX/V78678r+a5uZnYl53m+D6Hj0sTRX7nypHhnaoivNZecTaa7pXZd3hyyXxsonD7Tdy6vy6rY7
L83FZiafmbqL0lx2diTNtZV23n/LkzY2vyzNZfpm4OoQJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwR
JiwRJiwRJiwVeVbJw+Xjr6S5ttTPDG0jjaUF7eK/ejNBCCGk7pI0l01OpLl6bVPenW1/IM1dHv8i
n9nZ+16aq1c2pLnUe0XeXT3Ubty5GGxLc3xjwhJhwhJhwhJhwhJhwhJhwhJhwhJhwhJhwhJhwhJh
wlKMoZQfeDDe/1CaKw539A9Qiz/ob7WP2RalvFtVD7akuXy0L59ZjIbibu26dggh1Cu3pbn8+LF4
YpR3p0q77yCoD7iQNwNXiDBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhqciyOXm4rbQH7882
bshnlnvfSXOxqbW5+lLeXS/f1AbFuwliq/+mfbb+qnbmxVg+M59o1+qb/h3tvNGuvDsm7f+TyoE0
xzcmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLBVl3pOH88Pfpbm0uCaf2Vxbl+aq
H7UXE0zf/UTeHZL2sIX8ydfSXN3XH6ZfHO9Jc6nS/z+pq83GiyNprnz2SN59eeueNije6MI3JiwR
JiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJiwRJizFGLvyywFOH7wjzaXeS/IHKIba9femt6qdd/RU
3n15+21tsD6XxmL9XN6tXqdXX4oQQgjFWHvhQCorbXXRkXfn

There are no correlations among the guests. That's good, as we didn't put any in.

### Distributions of drinks

What did everyone drink?

In [40]:
@df x histogram( [:nM, :nW, :nSA, :nSK], layout=4, legend=false, nbins=15, title=["Mixed Drinks" "Wine" "Soda Adults" "Soda Kids"], 
    title_location=:center, line=nothing,
    color=[:green :darkgreen :darkblue :lightblue])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0 
 
 
 100000 
 
 
 200000 
 
 
 300000 
 
 
 Mixed Drinks 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Wine 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Soda Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Soda Kids

Easier to think about number of drinks per person (a kid is a teen or child)

In [41]:
@df x histogram( [:nM./:nA :nW./:nA :nSA./:nA :nSK./(:nT.+:nC) ], line=nothing,
                color=[:green :darkgreen :darkblue :lightblue],
                layout=4, nbins=20, label="", title=["mixed drinks/adult" "wine/adult" "soda/adult" "soda/kid"] )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 mixed drinks/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 0 
 
 
 100000 
 
 
 200000 
 
 
 300000 
 
 
 wine/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 soda/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.5 
 
 
 4.0 
 
 
 4.5 
 
 
 5.0 
 
 
 5.5 
 
 
 6.0 
 
 
 6.5 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 soda/kid

### Cost results

How do the total costs compare? Let's try a [Box plot](https://en.wikipedia.org/wiki/Box_plot)

In [42]:
@df x boxplot( ["cpr Package" "cpr Tab" "WT Package" "WT Tab"], [ :CtotalP :CtotalT :WtotalP :WtotalT], 
               ylab="Cost \$", ylim=(3000, 8000), legend=false )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Package 
 
 
 cpr Tab 
 
 
 WT Package 
 
 
 WT Tab 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 Cost $

Let's look at the distributions too...

In [43]:
histogram( [x[:CtotalP], x[:CtotalT], x[:WtotalP], x[:WtotalT]], bins=40, 
            layout=grid(4,1), color=[:red :blue :green :yellow], xlim=(3000, 6500), line=nothing,
            labels=["cpr Package" "cpr Tab" "WT Package" "WT Tab"])  # Note no commas in colors

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Tab 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 WT Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 WT Tab

Let's compare the drink packages to paying by drink (tab)...

In [44]:
h1 = @df x histogram( :CT_P, nbin=40, legend=false, title="cpr: How much more Tab is compared to Package", line=nothing)
h1 = vline!([0])
h2 = @df x histogram( :WT_P, nbin=40, legend=false, title="WT: How much more Tab is compared to Package", 
                                      xlab="Price difference \$", line=nothing)
plot(h1, h2, layout=(2,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -150 
 
 
 -100 
 
 
 -50 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 cpr: How much more Tab is compared to Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1250 
 
 
 -1000 
 
 
 -750 
 
 
 -500 
 
 
 -250 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 125000 
 
 
 WT: How much more Tab is compared to Package 
 
 
 Price difference $

Positive means that the price of putting beverages on a tab is *more* than the cost of the package. So using a tab is bad in those cases. Negative means that tab is *less* expensive than the package. 

In the above, we see that for WT, the tab plan is always cheaper than the package beverage plan. Beverage consumption does not make t

For cpr, the tab plan is usually better. Let's look at where the package plan is advantageous (right of the red line on the plot). 

### Cost - people correlation?

In [118]:
p1 = @df x histogram2d(:nA, :CtotalT, nbin=40, xlab="# Adults", ylab="cpr Tab \$")
p2 = @df x histogram2d(:nT, :CtotalT, nbin=15, xlab="# Teens", ylab="cpr Tab \$")
p3 = @df x histogram2d(:nC, :CtotalT, nbin=15, xlab="# Children", ylab="cpr Tab \$")
plot(p1, p2, p3, layout=(3,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 45 
 
 
 50 
 
 
 55 
 
 
 60 
 
 
 65 
 
 
 70 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 # Adults 
 
 
 cpr Tab $ 
 
 
<image width="462" height="82" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAc4AAABSCAYAAADdNTQ8AAAIWUlEQVR4nO3d206c1xnG8febHTMD
ODscBwMmxtsYQ1IbYohDLCdt1fYCql5CcgOVegc97C20FxBVPWqlVE4wBgqJzcYhdULYJAZvcOoG
sIeZYb4vB+lpK63nTZkM/H/njxYCxsvrYW0iAwDgQMokSir1Q38ZAADsZ0ycAAAEiOr9BQAAoNPq
VjOzSJwCWXECABCAiRMAgABUtQCwr+jVpdmuY07Y+8r0+3BajqZTRSnHihMAgABMnAAABKCqBYB9
pfEq01SqIGdbch1ytjN1Tsqx4gQAIAATJwAAAahqAeC/8uxQ1Xkq0yjKyllPZdqca5ez3anzcvZs
9rCcHW6rSjlWnAAABGDiBAAgAFUtgAax97Wp5x9IX2WqHcw3MytkX5Sz3el+OXs6rY87dLgmZ690
rcrZ032fSTlWnAAABGDiBAAgAFUtgD3kOZyvBjNqUr7L1MysOXdUznaleuVsr6OqfetIWc4OtK/J
WbUyNTNrfXtTzlZ7XpFyrDgBAAjAxAkAQACqWuBAarz7TNOpFilXyLbJY3ak9cq0P6tXtW+IB/PN
zAba1+XsK30LcvaZN/Rxaz0n5Wy1/ZScjWq7Uo4VJwAAAZg4AQAIQFUL1FXjVab1us+0Q3wCqj+r
jzn4glblmZld7V6Ws6cu3Jazxbf0mrd8bkjOJtkmPZvRs/mFMTkbzz6Ucqw4AQAIwMQJAEAAqlrs
I433BJRF+v9dfZVph5w9mjorZy86xn31Oa029VSmZ1+flbOFfv1gfnnwipyNW1+Ss9HOt3I2f1uv
TEvX9T8drMzqv4+37nZLOVacAAAEYOIEACAAVS1+ZOpUt7oO5jfL2UL2iJztSGu7TM3MfuKoTC+1
VeTsOyc+l7NdZ5akXPMv9J2xlZ4+ORu36rt5k6J+32zhxp/kbHVav6t2+R/6k2RTqz1ydvKR/vmb
2d6Scqw4AQAIwMQJAEAAqlr8D/V4Aqo+B/OLjkPynXWqTIcclenlrhU523P+jpxtvapVY2ZmlbMX
pNzuC/o9qOkt/f7VpoUJOVud0X+2d0YH5OzsepecHXuo3SVsZjbzRN/Nu2z6/bqbla+lHCtOAAAC
MHECABCAqnafizy7VD21aZSXci1NelV0JK0/LzSY1ccddFSmI8dW5OzpgTk5m9fP11vlxGtytvbc
y/rAouLU+3K2Mq3vyP1y8lU5O+7YZTr9jf4ni5tPHsvZVftUzm5W1uRsLX4iZ5NE++yy4gQAIAAT
JwAAAahqg9TrcL7jPtOoKGeLOc/h/PNSznOX6dBh/fD2YPtdOds7fFPOFgZ25Gzp0i/lrD3fq2e/
/UKO5hdG5Wz5mlbJLX2s/S6amU19dVzOTmzon73pkvbclZnZWrIoZ7fEXaZmZru1bTmrVqb/Se95
khUnAAABGnTF2XjXsqWinJwt5jrlbJe48jMz68/qryxcOaKt/oYc5wvPvPmJnM2+rm1mMjPb6b8q
Z5Oi/j1Oba7K2aaFcTlbHtM3zCx9rF9hN/31y1Lu+kN95TdX0jfLLNu8nN12bZZ5KmeTWF/5JY6V
n8/uns9jrDgBAAjAxAkAQADnErcOV7KZ+R7/jfQzTgXHZpnOtF5RXXBsmBl2bJgZOaY/AHxyUKup
8oN6VVQa+LmcTQ7pm0DSj/Tza9m1f8rZ6t//JWeXpvTfx4mVE3J2fEP//M2UvpFyS8ktecynFX2T
TuypTE2vw5MklrM+e1+Z1gsrTgAAAjBxAgAQIHLVra7KVN/F6Hn892hW32V6KadfyzbUpleml7u1
B3zN9MrUzCz3U/21g/I5badpdEh/ySJ5rL+SULj5Vzlbuq7vtv78E8cu07Vjcnb0gV6ZzlUeyNm7
sf4z2q7ck3Kx60o2R2V6gHaZHjSsOAEACMDECQBAgCidekbuE/LZw/LAnZl+OXvRcSHAsKMyvdSp
X0fVOzIlZ5te0yufct+wnK09q+80TW1qV9g1LeqXGJRH9Z/t4pT++zjhuJZt7KH+J4vb5UdydiWZ
lbNPxMrUzHc435KaGtSHlJNmVKb7FytOAAACMHECABAg6m/+jdxG9OVelAe++pJe2YycvCNnjw3e
lrOpn3XL2erR03I2btV3T3oUZv4sZ6sfaXd8Lk7qjySPOQ7m33BUprcc1eV6rF+AUK/7TPXKtF47
TalM8cNixQkAQAAmTgAAAmTe6zwkh0eO67Xnicv6/ZHpN9vkbPncr+Ws5wmozLq+qzY/80c5u/VB
q5yd+1Svl8fEJ6AmNrLymDPVdTm7VtPvmy1V9ftMa47D+ea4k5TD+YCOFScAAAGYOAEACBDt/P6Q
3tm8c0aOVnpG5Gzx+QtytnT/Qzmbn/2bPu4H+rf5i5v6/bpTjvtMxzf0naZTZe0ChHVHZfrUUZl6
noDyVaZVqkugwbDiBAAgABMnAAABos3Hv9WfFcs9q4+8tSxHC9N/kbNPRzNydtFRmV5b7ZGzkxv6
1zxT0+/XXa/qu6Z3qto9qnGyI49Zv4P57BQFDhJWnAAABGDiBAAgQCYq6U8TNd16X85WP/y3nJ0f
13fVfuSoTEcf6Ifz5+Ov5Oy96oKcLVUfyNk4qcjZJNmVszoqUwD/f6w4AQAIwMQJAECAaPt37fJ2
wvkbF+WBx9e65Ozko0bcZeo4nJ+U5Wzi2GnqQ20KYH9ixQkAQAAmTgAAAkR/OPWuXNVeu69XpnPJ
l3L2fuUzOVvZfSxn46QkZxPHfabUngDw48GKEwCAAEycAAAEiI63/Equau+V5+WBKzX9AoQk1u8z
5U5SAIAHK04AAAIwcQIAECBKpVrk7jKJHQfzXZWpB3UrAEDHihMAgABMnAAABIjMMnXqTKlMAQCN
hxUnAAABvgMhhSdgNCETyQAAAABJRU5ErkJggg==
" transform="translate(58, 12)"/>
 
 
 
 
 
 
 
 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 # Teens 
 
 
 cpr Tab $ 
 
 
<image width="462" height="82" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAc4AAABSCAYAAADdNTQ8AAAEJUlEQVR4nO3dXY5UVRSG4X1OnaKL
8Gs6UURFGkLEVkfglSNwJjoLJ+FwvPBSg6hEDQqBYBAhyE/TXV2nHIEXa12w3MnzjOAN6fTHDpt9
hgYAJaZtdUHGWB0AAD0xnAAQYDgBIMBwAkCA4QSAgKE6APgvfd447NnQ66/Eoc8z0DAsqxNS+vzT
BoAihhMAAgwnAAQYTgAIGFxAeP1cQHi9hk7/fjiOp6oT0lbTG9UJKWeXF6sTU

Looks like it's the number of adults that matter. 

In [128]:
p1 = @df x histogram2d(:nM, :CtotalT, nbin=40, xlab="# Mixed Drinks", ylab="cpr Tab \$")
p2 = @df x histogram2d(:nW, :CtotalT, nbin=40, xlab="# Wine", ylab="cpr Tab \$")
p3 = @df x histogram2d(:nSA, :CtotalT, nbin=40, xlab="# Soda Adults", ylab="cpr Tab \$")
p4 = @df x histogram2d(:nSK, :CtotalT, nbin=40, xlab="# Soda Kids", ylab="cpr Tab \$")
plot(p1, p2, p3, p4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 # Mixed Drinks 
 
 
 cpr Tab $ 
 
 
<image width="162" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKIAAACUCAYAAAAQ0fKrAAAK0klEQVR4nO2dWXPbyBWFuwFQMk1q
l2hpxrIWb+OZSlyZ/IP8hvzSPOct7zNVk+TBSWyJsmVZ+8ZNIpY8pIrd52JECCJt3XjO94Q7TYAA
ddyFM/f2bWuIIMowju393Mdvi+C+b4AQYyhEogQKkaiAQiQqoBCJCqL7voEvg3TCkjLOuOhaGpHP
p+//DHBGJCqgEIkKKESiAgqRqODeX1LvzvhMw/AfocRPZMVns1FuUZ4r7yMbEo2TL2N0OCMSFVCI
RAUUIlEBhUhUoMislDMftpSJkP/erIiCGz9rTVjmtgTykaTB8OIsFWMYF10Lh4q+d5yMYlbc35wz
IlEBhUhUQCESFVCIRAUUIlHBF65HvL0zzrlimT7z/g1ZG4qPVsS1MI6CBzfG8lqhuFZoJyFOs747
NgmMJd6YMcYkaQ/izHPKSSbH8FqpuJbJYgzBC+O5RelBe+PIqBSlA13MGZGogEIkKqAQiQooRKKC
MZiVu9cFwptrURrO4q2GQc07RvMxGU5BXA3nIK6bBYzTuvusQTMSiH+robivrvXNCqblOkEH4ovo
COJe2hocX6UXMNZPWhBn6fA0nfXMSyZ/S5E+zKdHx2VR7l6ryBmRqIBCJCqgEIkKKESigjGYFf+F
dLhxGZYtkXV/VmQwovAhxNVocXA8FTZgbDF9BPGqnYV4dgK/65HndeoRvthHAT7SVYLPcN6vuuNr
GDKHV2iaDjxTZIwxh8GnwXFRfWUmDEciag4z69c2ij9DblGXrHUsw7DFVHevTeSMSFRAIRIVUIhE
BRQiUcFnLQMrenP1DYo0JxWRHalV0JA07MbgeDVbgrGNOpZuPZ/CEqqlB12IV2qXg+PqBDqOLBPm
pFeF+Ljr4q0WGqooQFN03cZz49RleK4DvKeeyLQU4zyDND5ZQceJ4aNFnR7GA2dEogIKkaiAQiQq
oBCJCihEooI7uOYyrkmm9EQ7D29hUhRi+uthBZ3wol2D+PtwZXD8wyze0vPpS4hfLX2CuPHoAOLa
nHOok3N4bnsPaxcvT2cgfr+/PDhOMny+VowueirCn3s/dqk2ufCqOFcm/gyeM863K5GUqUccZyP8
m+GMSFRAIRIVUIhEBRQiUcEtzErRghg3nntrLeg1aO3E4LgS1mBsPliF+KlZhni9br1jXKT04+o2
xN++2IL44Saal2DB3Vd2iaYhmMSU33UXF2r1E3duJ8HnuxDNGS5j7M7QC1x3hyTD75FdImSnB2lI
MC5K6ZXJ0n2ZXak4IxIVUIhEBRQiUQGFSFRwC7NSph5NvtfKbg0TEPsGpRZiBqORLkL8pI63+mLa
1e/94Zv3MPb41VuIa68xk5I+QSOUpe5F3z44wbFd/N6eMCu92GWHzq/xs+d9/KnODNYcts2Zu05y
DmNxggYsy7WlS4bHODhkTAecEYkKKESiAgqRqIBCJCqgEIkKRl7Fhz0Oh2wtZvI9DqPArWqbMysw
1ohwxdtyFVNaz2adu11Z/Qhj1Wfoks0Cthwxoi4Q+gkeorPtHuLqwcOTeYg/dpzzf9/BFN/R9RXE
x8EhxK3Exf2kDWP5xu+yebtwyf5vn9tObZxwv2byFUMhEhVQiEQFFCJRwR3qEYdRZE5k0/XpwfFM
Og1jyzW81nodm5s/XnKGZOYZpvjsGhqK683fQRxeoJkJP+wMjuNTTEMe76FZaZ5hKnKn7Z7poIcm
4chi2u4yO4a4n7o0XpKhscmMNCcF+zfD4qmylDEcn6c+kTMiUQGFSFRAIRIVUIhEBWPoj+g3ZJed
HTDT4GdSjDFmzrpsSiPCrggbdXxZ35zFOsHG+u7guLKOn+2vvYQ4mcGsTXiOZiU7dlmNi3cbMHZw
jMbnfRvvc7fjnvlDjD0Nj4N9iHvxGcTXsesqkQmzkm+4LmLZzH2oRSmqKR1PQ/ZR4IxIVEAhEhVQ
iEQFFCJRQWmzku/m4P0XYU5C0aB9MsSWboteM/PVKTx3Ywpf7Ncef4C49tSVfsUvMHOS1NBgBC0s
v7LXWGJ11XT3dfQRd63aPsdrfejiT7bXc4uajgPcBtcv8zLmV7a+9QyKXByVy6SUMieS23fruC84
IxIVUIhEBRQiUQGFSFRAIRIV3CHFJ31z4I2g850Q25jVA6zlWzAu5bdewzTdqkjpza/jAin70rUk
iRcew1gWiDrIS6wDtG+wX2J336X19k5kvSGm9PY6aDAPMpfWaxm8Z7/e0Bhj4hQXZvkLovIuWS6A
GsXYFqXt7iet58MZkaiAQiQqoBCJCihEooJCszI0pWeMsV5aLxCLox6E2GGhkWJd4ErNnbtWFym9
tR2Iq6/QCFxt/mlwnDX+iPd49gbCaG8bz23iQq33W08Gx2/PMKXXbKEB2+1jR4bDwNVFdmKxOCqX
0hN7QZdowF6O+zcfZeGMSFRAIRIVUIhEBRQiUcGvmJWi2rSbW89VQsxC1II5iBsGd5fa9BZIbSxg
Ld/s5i7E6bOnECfzmy64QpMQtUTz9iYaobPt7yBunrgszdsWGq5mB+sEjwI0Te3Efbe/GMoYY9JU
dG8QreXAoIzcSu7/z6D4cEYkKqAQiQooRKICCpGogEIkKrhFPeLwbc0Cr41IJUDXvJBib8EnYqXe
d7POcW6+/A+MTfyIn+28/jPED+vPB8e9LtYqVnb/AnG3iTWGzR3cAq3ZcnWTO218voMMnfCpwe/y
ty5LMllvWLQyL/HGxsnnabj+OeGMSFRAIRIVUIhEBRQiUUFUmNLL7SaFJiLwGrZPBrhYaj6rQ7xS
xd2SGnW38OjhMqbpkkdr+L3n7yDuxK5tyOS//wpj/b9hSq/5y2uIf9nHusifT1wD9+YV1hvuB9go
vt3HNiL+vspZWmRORknjlWkTot+cSDgjEhVQiEQFFCJRAYVIVBCV7ZVnbSBiZ1aqFvsfzoQVjCv4
Ml+vukxEMIljwTE2Qq9EuCNUuOcMSvIPNBgffvo9xH/ffYLxGdYcbnfdfewE2zB2GeN9DFsQJXeL
ytuRMvZEf3eGccIZkaiAQiQqoBCJCihEooJcZiW3e1RBHFpnSAKh677IJLRjzMocXrhOEDM/4YKm
2U9iAZRojN4+dtmRrR00I/88wvKzn0/RnLxpYTP3rcC1qTuLsWn8VXyK95HiuanfvSG3W5RswD6M
ItP4dZkTCWdEogIKkaiAQiQqoBCJCihEooKRm7kn3gKhrsH010mCLTe2Wri4aiJYGhzvt7F2cXZr
E+JOH9OFhz23aOtfF+iK37Ww7rGZYDuTT3Yb4pZXY3gVY0uRVDZglzWGQ3oc5l3y111TOAqcEYkK
KESiAgqRqIBCJCqwhfWHsh9igHWBgXUGZGriGxibDpchXkm/hXjB6xIxXZF7PeN9dGK8zSNv8dSR
xcVSJ3ZPnIsGpCfixDMk+YbraHzK7Zv82zIco8AZkaiAQiQqoBCJCihEooLSZsVYNBXWq0eMRFu6
SDR3nwxwxye/jV3FTsJYIkzCdYoLpPwtaOWCpvx2tGhAUtEuDrIjpcxIETQrt4UzIlEBhUhUQCES
FVCIRAUUIlFBoWvOnyBdtPXGbnbU/0PoHs7F68rtwoY3Qh/eh7DcajpJkfP1fz+

In [129]:
p1 = @df x histogram2d(:nA, :CT_P, nbin=40, xlab="# Adults", ylab="cpr Tab-Pkg \$")
p2 = @df x histogram2d(:nT, :CT_P, nbin=15, xlab="# Teens", ylab="cpr Tab-Pkg \$")
p3 = @df x histogram2d(:nC, :CT_P, nbin=15, xlab="# Children", ylab="cpr Tab-Pkg \$")
plot(p1, p2, p3, layout=(3,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 45 
 
 
 50 
 
 
 55 
 
 
 60 
 
 
 65 
 
 
 70 
 
 
 -150 
 
 
 -100 
 
 
 -50 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 # Adults 
 
 
 cpr Tab-Pkg $ 
 
 
<image width="462" height="82" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAc4AAABSCAYAAADdNTQ8AAAJvUlEQVR4nO3dS3IjWRnF8XtTqYdf
kiU/qwpXdEQ1A7qnQATBKtgF62AH7IExG2ARjBhAdDRd/bC7bMmWZMl6pTIZMGPU9xzaWSr/f/Mv
biot+cY9+vRlDCGvAn6iIuq1zn121t013CcAH7es7gsAAGCXsHECAJCAjRMAgAQxhmYt33FWga9W
P358ZwgA/4sTJwAACdg4AQBIkMeYy8VO3BqtqNaorfRa7/U6qadzzY6X9FMlfmr08VPvM/cY/1+c
OAEASMDGCQBAgtjKX8kxU1mt5YWrqtBrw1au9aJaY93auojribXrQKf2LqgjNq3rKwci4k8VJ04A
ABKwcQIAkCDvtd/KxcvtWK4tyoVcW1WlXLs14uWy2si1odJjXi+a1u9VFetIuPTr9TqXDbV1atdj
9zq16+qY3sWIuK4O8d3qTOfECQBAAjZOAAAS5L+Nv5aLR5ketz419Np5nOnrholcuy7nRq1+zaXR
gVyUS2NdPZquxGharQshhCro98mLtJ14uaY0z3i99bWKqveqnsi0vh7vXYxbHc//ejlxAgCQgI0T
AIAE+R+uVnLx/aoj196uDo11z+Ta4UqPAmdGFDjKpnLtMup/o1l8kGs3lR6nF2L38saIwzdGp/a2
1O/xS4uXrXWNrmk9/KxnCMguTj+odnI+9fPHy5w4AQBIwMYJAECC/DdX38jFq3VLrr2e9OXau8W+
XDvfNuTam4X+eodLPV6eFXp6MtlcyrWPlR5fPsYnqW6a60M11kFbM4QQlqUepRdGzFtUetfztqxn
VnRpvC+c4SXqcAxrTSfSNtQTaYdQy8wTmx4vq4WcOAEASMDGCQBAgrj8++/lY27jx/f6wvdGt+dX
bbl28s1rufbxvifXTmd6F/G396dy7YPR+TxZN+Xa26VWOzMa5O6Wes50t9aHPSyNzth50DuBp5ke
az8FPZp2uohXRtf0SpyNXRrXW9ujE61O7eePw/+7cF2xtpMva9fMiRMAgARsnAAAJIhF9Rf5nPs0
+0pfeTmSS+PmUa5tPHwr17a+/odcW33QZ+QWH/RoenWrdy8//HAh147HWqw9N6Ll741O7anRIT5c
6bV3K73L+05vyA1PRqf2uNDjy8egd+ROM+0ztAr6jZpX+v+prTHr2ZmLXRlDG5xoerPVr9npBK5j
AAknTgAAErBxAgCQIBZ/PdF/PNo71ldeGY+7OtV/1B8KPT6JW6N2qke14fperzUa3YrbPbl2MzmQ
6pYPXXnN6Uh/Pz496UM1rscDuXa81O/xbKN3PTvx8s0il2u3RgPk7UIrHhd6d+vEGFCxCnrsOcv0
r6OcRydunNdrPTpR/9/qzJneio9d5MQJAEACNk4AABLExZ8GcniSHejH+uxXegdkcfW5XBvX+rF+
8/oLudZ6FNPeuVzb6Oi1TqdbOfmXVBeXeszUmFzLtfnwRq4N73+QS7cf9Mi0eNTj5cVQH+Zx+50+
RGQ61yL8EEIYzo+kunsjDh+u9I72u6UeaU82+iCQ0Ur/3C63+v+p0VbfD56MbutH55GNQYuXOXEC
AJCAjRMAgATxz7/8o3yuP+3oR/Pzfb0D67ynz+js9/Xaw1N9vm7TiLXznn6v8nM9AoldPUYMHW2Q
QbWvR3mh1H8IHbZGrfGD8/igR9PlRL/mYqjHvMVU/xutjHj5cah9vTOb63Oip3P9eidLvfbO6PKe
bIyBHOKM6RBCGK31eHmsNyCHhREvP4od15w4AQBIwMYJAECC/G83+qzMk7b+Y/XPDvT4pDs6k2vP
jXj5zZEeq/UP9Lj18vWPcu3+SI+m997dyrWZ2qzd0Tsgy2P98WvbrlF7pM/0tR7FlOsdnyHTI7ls
fifX7huvt/vha6kuG38nr+lE6cX3enS5edD/P86u9f+P45H+a4el0YF8O9WHl9w9GZ3a4jVz4gQA
IAEbJwAACeL5we/klsBW1B8B1a30aKxbaT+EDiGEs4YeBQ5aeqw9MFK1QUvvnrzo6O1qV0f6D4sv
e1oHcq+nr3n6To/ksqY+z7T1So/zsrf6IILNW30QSNXW462qqX+GKiNeLrtXUl1c6NFyXOkzY7OV
/vVM3Orvx/z9P/V11/r/i+2/n/Taqf6eWg71mHc+0j5/nDgBAEjAxgkAQILYyLpyVBujHrs0jK6+
RqZHxK1M71bLjdfbyYx4uXwl1x4EPQLpN/TXe9bWYu2u/rYIl3t6vHXa1iOq8z09ojo34vCj/blc
2z/VH1WXt/RHQB281mPTTPzaIb8whoC0jbNFacynfqU/OjFO9E76sNQ/B9XCeL0LuTRUhf43Kud0
1QIA8LNj4wQAIEGMsWU8k13fd6NTG/XH9cSoZ4HOunmmx56thj5oopXpMy/bUY+194PWrXa81Tvk
+pkeS5+19b+tEy//Yl/vmD7r6BHkhREvXw2Gcu3Rod5p2jvTOrX3zvVYujnQu2obb/T3VHmhR7Uh
19ddX30p18aNkbcasfa2/5lcGwvtmjlxAgCQgI0TAIAEMYRc76p1l5ZLvZXlZYM+AMGKtWM962aZ
/niiTIy1mw29+7htdC7nUX+th/FEru2X+mzQXtBj+LOWni+fdfTP30lbj+RO21o370lbj7TfdI3H
EO7p0eVgoMfLg7c3cm3rRO/yzo70jtysa/xPP9a/yiqu3kl1nDgBAEjAxgkAQAIrqn1paoumrWWd
dZ2uaXFdI5aOwem21mudSLuV6TNjmw29NjO+dtjL9Pm63cqItStt3UOjk77f1N8Xg7b+2Ts2fuxw
0dEHVJwZj13cz/V1L41IvGd0ah92ta5pTpwAACRg4wQAIEFN+aHLiZcL4zW/rFi7lmjaipb1Wmcg
hxMvOzIjgsyMucvOus2G3gmszpl24nDn8Ye9Uo+0T43HHx409Pdj34iXT4x4ud/WB4FcGvHymyPt
kYCcOAEASMDGCQBAArOr1ok9d9Eu3qvdipd38w1VV8d0PbOirUjcmjOtdi/r1+vMmHa6np3HH7aN
aLoT9CEi/VLvmD4K+qMiew29q73f0j4HnDgBAEjAxgkAQILdTMawA9SImEj7OezmnGmt1uuYNmqN
d5Qzn1qPtENoZHpk6nRM50aXtzVEJGrdy5w4AQBIwMYJAEAColp8QnZxMAYDOX6quh4mqJfWMyfa
4cTa0RiM4dRmRqzdyIhqAQD42bFxAgCQgKgWwDN6WfFyHeRH+oXgxcuV00Zc00AO8ezIiRMAgARs
nAAAJCCqBXbWS5udXMc10zH9HGobyCHixAkAQAI2TgAAEhDVAsAn5WXFvA51A+TECQBAAjZOAAAS
ENUCAHbY83c+c+IEACABGycAAAn+A1z+74CZjf4uAAAAAElFTkSuQmCC
" transform="translate(58, 12)"/>
 
 
 
 
 
 
 
 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 

Looks like the cost is driven by the number of adult who attend. 

### Explore cpr where package plan is favored

For cpr, the tab price is usually cheaper than the package price, and so the by drink (tab pricing) is the better option.

There are some parties where the opposite is true ... the package price beats the tab price (the positive part of the plot). What fraction of parties have this situation?

Here is the plot again

In [131]:
h1 = @df x histogram( :CT_P, nbin=40, legend=false, title="cpr: How much more Tab is compared to Package", line=nothing)
h1 = vline!([0])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -150 
 
 
 -100 
 
 
 -50 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 cpr: How much more Tab is compared to Package

In [45]:
ctc = @where x (:CT_P .> 0);   # ctm = "cpr tab compare"
percPkgBest = @>> nrow(ctc) / nrow(x) *100  printfmtln("{1:.1f}%")

16.7%


So for a little more than 16% of the simulated parties, the tab cost is less expensive than the package cost. What's special about those parties?

In [46]:
@df ctc histogram( [ :nA, :nT, :nC ], layout=(1,3), legend=false, nbins=15, title=["#Adults" "#Teens" "#Children"], 
    title_location=:left, line=nothing,
    color=[:red :darkred :pink])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 #Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20.0 
 
 
 22.5 
 
 
 25.0 
 
 
 27.5 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
 
 25000 
 
 
 #Teens 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
 
 #Children

Hmm. These seem to be parties where there are few adults and lots of kids.

In [47]:
p1 = @df ctc histogram2d( :nA, :nT, nbin=15, xlab="Number of adults", ylab="Number of teens")
p2 = @df ctc histogram2d( :nA, :nC, nbin=15, xlab="Number of adults", ylab="Number of children")
p3 = @df ctc histogram2d( :nT, :nC, nbin=15, xlab="Number of teens",  ylab="Number of children")
plot(p1, p2, p3, layout=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 Number of adults 
 
 
 Number of teens 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAEgElEQVR4nO3dzY4UVRjG8bebAhxF
SRRiNEGWbr0Fl16DN8QVsXTnSlwSE4JByAxDd01/TVfXd53jggvgqWTQh+T/W7+pM5P806s35yye
/fxLDtFPv/0qzaXfn6ifjPjzpTRW/vGjNPf6zQ/y0X+V30pzf1dn0tw/1UI++7LrpLmL5Vr+5i6/
lea66Vqa69NJPnsQvxl5ksaW8snAf4gwYYkwYYkwYYkwYYkwYYkwYYkwYYkwYYkwYYkwYYkwYYkw
YYkwYanoxtvycHP+VJo7W1/K3zy+/F47u/pCmiurL+Wz94P2v287bZ1t24/y2ZvFQZqrQ5uLiBiz
tkqnrr2lrP8/kZM2xtobPmWECUuECUuECUuECUuECUuECUuECUuECUuECUuECUuECUuECUuECUvF
1emePHyrKqW5dK7fElbvHklzF2vtZrZVra3HRUSs20L7ZqutdG1yLZ/dLrUVtTrt5W8OqZHmsrjO
lpL2N0ZE5NDW2SK0ywX5xYQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoSl
4tjflYfvPH8mzTUXX8vfvFo/kOYuq6+kuTenz+Sz1612i9tp0tbEqkUln33IK2luEm9wi4jo5Vvc
BvGL6ipb6Le9hTbHLyYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsFYde
f+R0eq2tYF1fPJa/uRMfJV01Z9JcPWmrbBERh169xU27ve643Mhnj6mV5rpRW2WLiEhJW2fLN/wg
aUREFm9xU9fj+MWEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEJcKEpWLMepunV99J
c9urb+RvvtprF3C9a7QtqJX2nGJERJSjti21XV5Jc8OMC7Da6SjN6RdgRWRx9qbffXxP2xriLUl8
0ggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTloptV8jDp+19aW5/1C7KiojY
9XekuW2vXZa16bV3HyMiDuLbj3U+SHNt0i/AUt+ITDNW6dR3GkO8LEtfj4uIPGdF7sP4xYQlwoQl
woQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoSlopnx9mK5eijNvdg+kL/5rtHW7jad
tla1y/p1b9VSW1NT333sJ22NLiJinGppTn338f2wus52sytqc6gn84sJS4QJS4QJS4QJS4QJS4QJ
S4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJS0XZ6mtv5zvt8dJVe1f+5mWjnb8etHW200JbJ4uIuM6l
NNcnbZ1tSvrNbOqjoHNuXPs4j5fqp98kfjFhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBh
iTBhiTBhiTBhqehmXCZWNp9Lc7v+lvzN/aCtatWhrZQdFhv57HHSvjkkbeUuh/7AasqDNjjrtrcZ
s8rn/sdb4fjFhCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChKV/AexoU72jU4n5
AAAAAElFTkSuQmCC
" transform="translate(55, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAACUCAYAAACeJ3mSAAABW0lEQVRogd3a23EDQQhE0V57nH8S
ztEvBpzE+aCkAG7RFxitdvRUf07A5/SU4OT0/UagRqBZF20UKAzUyFHqC4FUtOf+GFD6GtBTvwh0
EWhhtPpTIOaoDWifozCQc8TOo23RUmjXcpkjVdE+R0ZRDnqGyIlRlBOjKGfqQaD7ZkDpdRXVuwHN
qGiFos1V0RY6UhWpOWom260IqqiVo2kme1vXnCO1Ii6a6tplK+IcqTlSjli0y3ZtH4i1n012o8l2
XdvnaJhs5ohwYEXMEWv/XXceQUeEk9Nsstd1beEcsZ8QBJOcUg/s+xzB9hMOjFZsjlw0BHLRCCY5
pV6NwRUhHPkQ8bog6MiQFjqCxwhyVEGgYRWtc3TRS//TSnYNqohFKwUaVtGDLqEK3UItjHZVtAq6
px10L+a6dlW0ZiC1/e5fYy8cbVg0doygFdnnKIOijZqjQcfIk3yQL/9/fmk1aCz3ydQAAAAASUVO
RK5CYII=
" transform="translate(232, 12)"/>
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
<polyline clip-path="url(#clip7500)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 250.315,160.065 250.315,143.971 256.315,143.971 250.315,143.971 250.315,127.747 256.315,127.747 250.315,127.747 250.315,111.523 256.315,111.523 250.315,111.523 
 250.315,95.2994 256.315,95.2994 250.315,95.2994 250.315,79.0755 256.315,79.0755 250.315,79.0755 250.315,62.8515 256.315,62.8515 250.315,62.8515 250.315,46.6276 
 256.315,46.6276 250.315,46.6276 250.315,30.4037 256.315,30.4037 250.315,30.4037 250.315,14.1797 256.315,14.1797 250.315,14.1797 250.315,11.811 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 Number of adults 
 
 
 Number of children 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAE5klEQVR4nO3dS24cZRhG4a+bCiHc
FAFCgjGINSCxB0YsiR2xA0+RmDEKshzbsdvui90XV9+qigEs4LQUkTfiPONPVXJ00pP/UqOzn34e
Cvrx7Bc015/9Sh9Z9cc5Gpv+/gOae3P9DX71n5Nv0dyr1cdo7mIzxu++bHdobjJa4GfORzdobnOc
oblD/4TffehWaK4f9miO/0tK/yHDVCTDVCTDVCTDVCTDVCTDVCTDVCTDVCTDVCTDVCTDVKTm0H2A
h9vL39Dci9k9fuZ28hLNbVaform7JXteVdXq8AzN3e/Y/9/prsPv3tYRzbWjNX7mrt+guW5gG0i6
ns1VVQ0D+3uq2J4hfzEVyTAVyTAVyTAVyTAVyTAV

Indeed, these are cases where you have the minimum of adults and the maximum of kids. Let's look some more. Are we picking the parties that are skewed
to low drink consumption among adults and high consumption among kids? 

In [48]:
using StatsBase
function siHist(v1, v2, nbins1, nbins2, title)
    """
    Superimpose two histograms, one filled and one open
    """
    
    p = histogram( v1, nbins=nbins1, normalized=true, label="", line=nothing, title=title)
    
    f = fit(Histogram, v2, nbins=nbins2, closed=:left) ; f = normalize(f)
    p = plot!(f, seriestype=:step, normalized=true, label="")
end
    

siHist (generic function with 1 method)

Let's compare what these people drink (filled histogram) to the whole set (open histogram)

In [49]:
p1 = siHist( ctc[:nM], x[:nM], 15, 15, "Mixed Drinks")
p2 = siHist( ctc[:nW], x[:nW], 15, 15, "Wine")
p3 = siHist( ctc[:nSA], x[:nSA], 15, 18, "Soda Adults")
p4 = siHist( ctc[:nSK], x[:nSK], 15, 15, "Soda Kids")
plot(p1, p2, p3, p4, layout=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 Mixed Drinks 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 Wine 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 Soda Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7704)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.4014,375.813 45.4014,375.812 63.4716,375.812 63.4716,375.521 81.5419,375.521 81.5419,368.239 99.6122,368.239 99.6122,330.725 117.682,330.725 117.682,280.337 
 135.753,280.337 135.753,263.45 153.823,263.45 153.823,279.5 171.893,279.5 171.893,318.142 189.964,318.142 189.964,354.957 208.034,354.957 208.034,371.571 
 226.104,371.571 226.104,375.343 244.174,375.343 244.174,375.788 262.245,375.788 262.245,375.813 280.315,375.813 280.315,375.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 0.000 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 Soda Kids

Again, it's easier to look at drinks per person

In [50]:
p1 = siHist( ctc[:nM]./ctc[:nA], x[:nM]./x[:nA], 15, 15, "Mixed Drinks/Adult")
p2 = siHist( ctc[:nW]./ctc[:nA], x[:nW]./x[:nA], 15, 15, "Wine/Adult")
p3 = siHist( ctc[:nSA]./ctc[:nA], x[:nSA]./x[:nA], 15, 18, "Soda/Adult")
p4 = siHist( ctc[:nSK]./(ctc[:nT].+ctc[:nC]), x[:nSK]./(x[:nT].+x[:nC]), 15, 15, "Soda/Kid")
plot(p1, p2, p3, p4, layout=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 Mixed Drinks/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7902)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1504,175.813 39.1504,175.772 55.6462,175.772 55.6462,174.878 72.142,174.878 72.142,166.041 88.6378,166.041 88.6378,131.855 105.134,131.855 105.134,69.9334 
 121.629,69.9334 121.629,31.4961 138.125,31.4961 138.125,65.8586 154.621,65.8586 154.621,124.41 171.117,124.41 171.117,157.454 187.613,157.454 187.613,171.748 
 204.109,171.748 204.109,174.874 220.604,174.874 220.604,175.696 237.1,175.696 237.1,175.8 253.596,175.8 253.596,175.812 270.092,175.812 270.092,175.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Wine/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7903)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 319.043,175.813 319.043,175.813 335.987,175.813 335.987,175.799 352.93,175.799 352.93,175.534 369.874,175.534 369.874,172.878 386.818,172.878 386.818,158.642 
 403.761,158.642 403.761,116.635 420.705,116.635 420.705,70.2823 437.649,70.2823 437.649,31.4961 454.593,31.4961 454.593,87.2949 471.536,87.2949 471.536,137.895 
 488.48,137.895 488.48,165.759 505.424,165.759 505.424,173.955 522.367,173.955 522.367,175.552 539.311,175.552 539.311,175.796 556.255,175.796 556.255,175.813 
 573.199,175.813 573.199,175.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 Soda/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7904)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.813 32.2221,375.807 46.2266,375.807 46.2266,375.717 60.231,375.717 60.231,374.868 74.2355,374.868 74.2355,370.614 88.24,370.614 88.24,354.131 
 102.244,354.131 102.244,323.681 116.249,323.681 116.249,257.374 130.253,257.374 130.253,231.496 144.258,231.496 144.258,239.063 158.262,239.063 158.262,285.475 
 172.267,285.475 172.267,331.099 186.271,331.099 186.271,357.823 200.276,357.823 200.276,370.853 214.28,370.853 214.28,374.657 228.285,374.657 228.285,375.567 
 242.289,375.567 242.289,375.781 256.294,375.781 256.294,375.808 270.298,375.808 270.298,375.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.5 
 
 
 4.0 
 
 
 4.5 
 
 
 5.0 
 
 
 5.5 
 
 
 6.0 
 
 
 6.5 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 Soda/Kid 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7905)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 335.517,375.813 335.517,375.813 349.618,375.813 349.618,375.808 363.719,375.808 363.719,375.745 377.821,375.745 377.821,375.341 391.922,375.341 391.922,372.66 
 406.023,372.66 406.023,362.885 420.125,362.885 420.125,336.991 434.226,336.991 434.226,294.83 448.327,294.83 448.327,264.492 462.428,264.492 462.428,250.714 
 476.53,250.714 476.53,293.656 490.631,293.656 490.631,336.204 504.732,336.204 504.732,362.538 518.833,362.538 518.833,372.746 532.935,372.746 532.935,375.188 
 547.036,375.188 547.036,375.74 561.137,375.74 561.137,375.805 575.239,375.805 575.239,375.813 
 "/>

Indeed we see that soda consumption is very high.

## Back Matter

In [51]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


When rendered correctly, this [Jupyter Notebook](http://jupyter.org/) uses the [Fira Code font](https://github.com/tonsky/FiraCode); a monospaced font with programming [ligatures](https://en.wikipedia.org/wiki/Typographic_ligature).

In [52]:
# Set up the notebook style
function css_styling()
    styles = open("custom.css") do f
        readstring(f)
    end
end
display(HTML(css_styling()))

HTML{String}("<style type=\"text/css\">\n    .container { width:100% !important; }\n    @font-face {\n       font-family: 'Fira Code';\n               src: url(\"https://cdn.rawgit.com/dunovank/jupyter-themes/1e851888/jupyterthemes/fonts/monospace/firacode/firacode.otf\") format(\"opentype\");\n    }\n   .CodeMirror {\n      font-family: 'Fira Code';\n      font-variant-ligatures: initial;\n    }\n\n   .cm-string {\n       font-variant-ligatures: none;\n    }\n \n    .input_area pre, .highlight pre {\n      font-family: 'Fira Code';\n      font-variant-ligatures: initial;\n   }\n\n   .highlight .err { border: 0px solid #FF0000 } /* Remove @> highlighting */\n\n\n</style>\n<script>\n    MathJax.Hub.Config({\n            TeX: {\n                extensions: [\"AMSmath.js\"]\n                           },\n                tex2jax: {\n                    inlineMath: [ ['\$','\$'], [\"\\\\(\",\"\\\\)\"] ],\n                    displayMath: [ ['\$\$','\$\$'], [\"\\\\[\",\"\\\\]\"] ]\n                },\n                displayAlign: 'center', // Change this to 'center' to center equations.\n                \"HTML-CSS\": {\n                    styles: {'.MathJax_Display': {\"margin\": 4}}\n                }\n        });\n</script>\n")